https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb

https://arxiv.org/pdf/1409.0473.pdf?utm_source=ColumnsChannel

https://happy-jihye.github.io/nlp/nlp-7/

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
seed=1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic=True

In [3]:
spacy_de=spacy.load('de_core_news_sm')
spacy_en=spacy.load('en_core_web_sm')

In [4]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
SRC=Field(tokenize=tokenize_de, init_token='', eos_token='',lower=True)
TRG=Field(tokenize=tokenize_en, init_token='',eos_token='',lower=True)

In [6]:
train_data,valid_data,test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC,TRG))#exts는 source와 target으로 사용할 언어를 지정

In [8]:
print(len(vars(train_data.examples[0])['src']))
print(len(vars(train_data.examples[1])['src']))

print(vars(train_data.examples[0]))
print(vars(train_data.examples[1]))

13
8
{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}
{'src': ['mehrere', 'männer', 'mit', 'schutzhelmen', 'bedienen', 'ein', 'antriebsradsystem', '.'], 'trg': ['several', 'men', 'in', 'hard', 'hats', 'are', 'operating', 'a', 'giant', 'pulley', 'system', '.']}


In [11]:
SRC.build_vocab(train_data, min_freq=2)#build_vocab함수를 이용하여 각 token을 indexing해줌. 이때, source와 target의 vocabulary는 다름
TRG.build_vocab(train_data, min_freq=2)#이때, vocabulary는 training set에서만 만들어져야함

In [12]:
device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:
BATCH_SIZE=128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)
#BucketIterator를 이용해 batch size별로 token묶고 어휘를 읽은수 있는 token에서 index로 변환

In [22]:
print(TRG.vocab.stoi['pitched'])#<pad>의 index=1

for i, batch in enumerate(train_iterator):
    src=batch.src
    trg=batch.trg
    
    src=src.transpose(1,0)
    print(f"첫번째 배치의 text크기:{src.shape}")
    print(src[0])
    print(src[1])
    
    break

        
print(len(train_iterator))
print(len(train_iterator)*128)

4284
첫번째 배치의 text크기:torch.Size([128, 36])
tensor([  2,   4,  12, 157,   8,   6,  13, 339,  33, 592,  27, 694,   3,   2,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1])
tensor([   2,    4,  958,   19, 4972,  211,    6,  196,  951,    3,    2,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])
227
29056


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embdim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim,emb_dim)
        #양방향=True
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True) #ht=EncoderGRU^->(e(xt^->),ht-1^->) 두개 넣음.
         # 양방향 rnn의 출력값을 concat 한 후에 fc layer에 전달합니다.
        self.fc=nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout=nn.Dropout(dropout)
    def forward(self,src):
        #src=[src len, batch_size]
        embedded= self.dropout(self.embedding(src))
        #embedded = [src len, batch_size, emb dim]
        outputs, hidden = self.rnn(embedded)
        #outputs=[단어길이, 배치사이즈, 은닉차원 * num_directions] 이건 time step마다 나오는 outputs임 //output은 언제나 hidden layer의 top에 있음
        #hidden=[n_layers*num_direction(2), batch size, hid_dim]
        #hidden layer는 [forward1,backward1,forward2,backward2... 식으로 쌓임]
        #hidden[-2,:,:]--> 마지막 forward, hidden[-1,:,:]->마지막 backward //hidden: 마지막 은닉상태 ht값임.
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) +dec_hid_dim, dec_hid_dim) #attn= 얘는뭘까 in= (enc_hid_dim * 2) +dec_hid_dim, out=dec_hid_dim ?
        #Energy 계산시 attn에 st-1과 H가 들어감. 
        self.v = nn.Linear(dec_hid_dim,1, bias=False) #[1,dec_hid_dim] tensor ^at=vEt
    
    def forward(self,hidden,encoder_outputs):
        #encoderoutput=(모든 h들) [src len,batchsize,enc_hid_dim*2]
        batch_size=encoder_outputs.shape[1]
        src_len=encoder_outputs.shape[0]
        
        hidden=hidden.unsqueeze(1).repeat(1, src_len, 1)#unsqueeze dim1에 1인차원채우기 torch.repeat(*sizes)
        
        encoder_outputs=encoder_outputs.permute(1,0,2)#차원의 자리만 바꿈 (0,1,2)->(1,0,2) 
        #위의 두코드는 아마 차원맞추기 위해서 짠듯 cat할려고 나중에 print로 한번 넣어보면 알듯?
        
        energy=torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))# Et=tanh(attn(st-1,H))
        
        attention=self.v(energy).squeeze(2)# at=vEt로 a 얘도 차원봐야알듯
        
        return F.softmax(attention, dim=1)#A 계산

In [ ]:
class Decoder(nn.Module):
    def __init__(self,output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        
        self.output_dim= output_dim
        self.attention=attention
        self.embedding=nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim*2) + emb_dim,dec_hid_dim)
        self.fc_out=nn.Linear((enc_hid_dim*2)+dec_hid_dim + emb_dim, output_dim)
        self.dropout=nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        
        input= input.unsqueeze(0)
        
        embedded=self.dropout(self.embedding(input))
        
        a=self.attention(hidden, encoder_outputs)
        
        a=a.unsqueeze(1)
        
        encoder_outputs = encoder_outputs.permute(1,0,2)
        
        weighted=torch.bmm(a, encoder_outputs)
        
        weighted=weighted.permute(1,0,2)
        
        rnn_input=torch.cat((embedded, weighted), dim=2)
        
        output, hidden=self.rnn(rnn_input, hidden.unsqueeze(0))
        
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output=output.squeeze(0)
        weighted=weighted.squeeze(0)
        
        prediction=self.fc_out(torch.cat((output, weighted, embedded),dim=1))
        
        return prediction, hidden.squeeze(0)